Required Packages to install

In [ ]:
!pip install langchain chromadb python-dotenv tiktoken openai scikit-learn

Importing Libraries

In [ ]:
import asyncio
import os
from typing import Any, Dict, List, Tuple

import tiktoken
from dotenv import find_dotenv, load_dotenv
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.qa_with_sources.base import BaseQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_core.callbacks import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import Field
from langchain_core.retrievers import BaseRetriever

In [ ]:
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
web_links = ["https://en.wikipedia.org/wiki/FIFA_World_Cup"]
loader = WebBaseLoader(web_links)
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(all_splits, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
context = all_splits
sys_prompt = """
Answer the question based on the context. If you don't know the answer, just say that you don't know. Don't try to make up an answer.
{context}
Respond in the persona of %s
Question: {question}
Helpful Answer:
"""

In [ ]:
class RetrievalQAWithTokenLimit(RetrievalQA):
    reduce_k_below_max_tokens: bool = True
    max_tokens_limit: int = 600

    def _reduce_tokens_below_limit(self, docs: List[Document]) -> List[Document]:
        num_docs = len(docs)

        if self.reduce_k_below_max_tokens and isinstance(
            self.combine_documents_chain, StuffDocumentsChain
        ):
            tokens = [
                self.combine_documents_chain.llm_chain._get_num_tokens(doc.page_content)
                for doc in docs
            ]
            token_count = sum(tokens[:num_docs])
            while token_count > self.max_tokens_limit:
                num_docs -= 1
                token_count -= tokens[num_docs]

        return docs[:num_docs]

    def _get_docs(
        self, question: str, *, run_manager: CallbackManagerForChainRun
    ) -> List[Document]:
        docs = self.retriever.get_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        return self._reduce_tokens_below_limit(docs)

    async def _aget_docs(
        self, question: str, *, run_manager: AsyncCallbackManagerForChainRun
    ) -> List[Document]:
        docs = await self.retriever.aget_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        return self._reduce_tokens_below_limit(docs)

    @property
    def _chain_type(self) -> str:
        """Return the chain type."""
        return "retrieval_qa_with_token_limit"

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


def qa(question):
    qa_message = [
        SystemMessagePromptTemplate.from_template(
            sys_prompt % ("AI Assistant"), input_variables=["context", "question"]
        ),
    ]
    qa_prompt = ChatPromptTemplate.from_messages(qa_message)
    chain = RetrievalQAWithTokenLimit.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": qa_prompt, "verbose": True},
    )
    answer = chain.run(question)
    return answer

In [ ]:
question = "Where is the world cup?"
answer = qa(question)
print(answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
Answer the question based on the context. If you don't know the answer, just say that you don't know. Don't try to make up an answer.
The FIFA World Cup, often simply called the World Cup, is an international association football competition between the senior men's national teams of the members of the Fédération Internationale de Football Association (FIFA), the sport's global governing body. The tournament has been held every four years since the inaugural tournament in 1930, with the exception of 1942 and 1946 due to the Second World War. The reigning champions are Argentina, who won their third title at the 2022 tournament.
The contest starts with the qualification phase, which takes place over the preceding three years to determine which teams qualify for the tournament phase. In the tournament phase, 32 teams compete for the title at venues within the host nation(s) 